In [1]:
!pip install langchain openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 9.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.5/76.5 kB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 5.3 MB/s eta 0:00:00


In [2]:
import os
os.environ['OPENAI_API_KEY'] = 'your api key'

In [3]:
import inspect
import re

from getpass import getpass
from langchain import OpenAI, PromptTemplate
from langchain.chains import LLMChain, LLMMathChain, TransformChain, SequentialChain
from langchain.callbacks import get_openai_callback

In [4]:
llm = OpenAI(temperature=0)

In [11]:
llm_math = LLMMathChain(llm=llm)
llm_math( "What is area of a circle with radius 3.9?")

{'question': 'What is area of a circle with radius 3.9?',
 'answer': 'Answer: 47.7594'}

In [12]:
def count_tokens(chain, query):
    with get_openai_callback() as cb:
        result = chain.run(query)
        print(f'Spent a total of {cb.total_tokens} tokens')

    return result

In [13]:
llm_math = LLMMathChain(llm=llm, verbose=True)
count_tokens(llm_math, "What is 13 raised to the .3432 power?")



> Entering new LLMMathChain chain...
What is 13 raised to the .3432 power?
```text
13**.3432
```
...numexpr.evaluate("13**.3432")...

Answer: 2.4116004626599237
> Finished chain.
Spent a total of 264 tokens


'Answer: 2.4116004626599237'

In [14]:
print(llm_math.prompt.template)

Translate a math problem into a expression that can be executed using Python's numexpr library. Use the output of running this code to answer the question.

Question: ${{Question with math problem.}}
```text
${{single line mathematical expression that solves the problem}}
```
...numexpr.evaluate(text)...
```output
${{Output of running the code}}
```
Answer: ${{Answer}}

Begin.

Question: What is 37593 * 67?
```text
37593 * 67
```
...numexpr.evaluate("37593 * 67")...
```output
2518731
```
Answer: 2518731

Question: 37593^(1/5)
```text
37593**(1/5)
```
...numexpr.evaluate("37593**(1/5)")...
```output
8.222831614237718
```
Answer: 8.222831614237718

Question: {question}



In [15]:
# we set the prompt to only have the question we ask
prompt = PromptTemplate(input_variables=['question'], template='{question}')
llm_chain = LLMChain(prompt=prompt, llm=llm)

# we ask the llm for the answer with no context

count_tokens(llm_chain, "What is 13 raised to the .3432 power?")

Spent a total of 17 tokens


'\n\n2.907'

In [16]:
print(inspect.getsource(llm_math._call))

    def _call(
        self,
        inputs: Dict[str, str],
        run_manager: Optional[CallbackManagerForChainRun] = None,
    ) -> Dict[str, str]:
        _run_manager = run_manager or CallbackManagerForChainRun.get_noop_manager()
        _run_manager.on_text(inputs[self.input_key])
        llm_output = self.llm_chain.predict(
            question=inputs[self.input_key],
            stop=["```output"],
            callbacks=_run_manager.get_child(),
        )
        return self._process_llm_result(llm_output, _run_manager)



In [31]:
def transform_func(inputs: dict) -> dict:
    text = inputs["text"]

    # replace multiple new lines and multiple spaces with a single one
    text = re.sub(r'(\r\n|\r|\n){2,}', r'\n', text)
    text = re.sub(r'[ \t]+', ' ', text)

    return {"output_text": text}

In [18]:
clean_extra_spaces_chain = TransformChain(input_variables=["text"], output_variables=["output_text"], transform=transform_func)

In [19]:
clean_extra_spaces_chain.run('A random text  with   some irregular spacing.\n\n\n     Another one   here as well.')

'A random text with some irregular spacing.\n Another one here as well.'

In [33]:
template = """Paraphrase this text:

{output_text}

In the style of a {style}.

Paraphrase: """
prompt = PromptTemplate(input_variables=["style", "output_text"], template=template)

In [21]:
style_paraphrase_chain = LLMChain(llm=llm, prompt=prompt, output_key='final_output')

In [22]:
sequential_chain = SequentialChain(chains=[clean_extra_spaces_chain, style_paraphrase_chain], input_variables=['text', 'style'], output_variables=['final_output'])

In [34]:
input_text = """
deep learning is at the core of large                                  language models,
                  enabling them to understand and generate human language.                These models have achieved remarkable
                   success in various language-related tasks,
                    revolutionizing
the field of natural
                       language processing.
                       One of the key advantages of LLMs is their ability to perform transfer learning. They can be pre-trained on a large dataset and then fine-tuned for specific tasks.
        This
          reduces the need for large task-specific                      datasets and accelerates the development of natural                       language processing
          applications.
"""

In [35]:
count_tokens(sequential_chain, {'text': input_text, 'style': 'a 90s rapper'})

Spent a total of 220 tokens


"\nDeep learnin's the core of big language models, they understand and create human language, they got success in language related tasks, revolutionizin' NLP. The key advantage of LLMs is they can do transfer learnin', pre-train on a big dataset then fine-tune for specific tasks, no need for big task-specific datasets, acceleratin' NLP applications."

In [38]:
count_tokens(sequential_chain, {'text': input_text, 'style': 'formal'})

Spent a total of 236 tokens


'\nDeep Learning is the foundation of large Language Models, allowing them to comprehend and generate human language. These models have been incredibly successful in a variety of language-related tasks, revolutionizing the area of Natural Language Processing. One of the major benefits of LLMs is their capacity to execute Transfer Learning. They can be pre-trained on a large dataset and then adjusted for particular tasks. This diminishes the requirement for large task-specific datasets and hastens the advancement of Natural Language Processing applications.'

In [37]:
count_tokens(sequential_chain, {'text': input_text, 'style': 'academic'})

Spent a total of 236 tokens


'\nDeep learning is the foundation of large language models, allowing them to comprehend and generate human language. These models have been incredibly successful in a variety of language-related tasks, revolutionizing the area of natural language processing. One of the major benefits of LLMs is their capacity to do transfer learning. They can be pre-trained on a large dataset and then adjusted for particular tasks. This minimizes the requirement for large task-specific datasets and hastens the advancement of natural language processing applications.'